In [ ]:
# Code to create the Ensemble with voting aggregation
# the Autoselection porduced by SOCP controls the pruning degree of the Ensemble

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
import numpy as np
from scipy import io
from sklearn.metrics import  accuracy_score

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# read the results of running the SOCP autoselection code
results = io.loadmat("socp_output.mat")
result = results["result_train"]
print(result.shape)

In [ ]:
# read model output matrix
data = io.loadmat("RandModel_PRED_m.mat")
pred = data["PRED_m"]
print(pred.shape)


In [ ]:
print(max(result[24]))

In [ ]:
# segment the data set based on the train validation split determined by SOCP process (40k samples in this example)
segmented_first = pred[:,0:40000]
print(segmented_first.shape)

In [ ]:
segmented_last = pred[:,40000:]
print(segmented_last.shape)


In [ ]:
# Ensemble run with a threshold hyperparamter to specify pruning degree
# use this section for experimentation on validation part to obtain the best subset
experiment_eval = []
for exp in range(result.shape[1]):
    print("hyper-parameter experiment{}".format(exp))
    #weights = result[exp]*100
    weights =[]
    for model_weight in range(result.shape[0]):
        weights.append(result[model_weight][exp]*100)

    # specify pruning degree threshold
    thresh = 0.131
    weights = [1 if ele > thresh else 0 for ele in weights ]
    larger_elements = [element for element in weights if element > 0]
    number_of_elements = len(larger_elements)
    
    
   # voting 
    voted_pred = [] 
    for model in range(segmented_last.shape[0]):
        pred_temp = segmented_last[model]
        if weights[model]:
            voted_pred.append(pred_temp)
    voted_pred = np.array(voted_pred)
    # aggregation
    aggregate = []
    for sample in range(voted_pred.shape[1]):
        sample_preds =[]
        for model in range(voted_pred.shape[0]):
            sample_preds.append(voted_pred[model][sample])
        
        aggregate.append(max(set(sample_preds), key = sample_preds.count))
    # save accuracy results to further analyse on validation set   
    acc = accuracy_score(y_train[40000:],aggregate)
    experiment_eval.append(acc)
    

In [ ]:
# evaluation checks on validation results
best_experiment = experiment_eval.index(max(experiment_eval))
print(best_experiment)
print(experiment_eval[best_experiment])

In [ ]:
worst_experiment = experiment_eval.index(min(experiment_eval))
print(worst_experiment)
print(experiment_eval[worst_experiment])

In [ ]:
average_experiment = np.average(experiment_eval)
print(average_experiment)

In [ ]:
print(experiment_eval)

In [ ]:
# full Ensemble results for comparison checks
og_model_acc = []
for model in range(segmented_last.shape[0]):
    pred_temp = segmented_last[model]
    acc_temp = accuracy_score(y_train[40000:],pred_temp)
    og_model_acc.append(acc_temp)

In [ ]:
print(og_model_acc)

In [ ]:
best_og = og_model_acc.index(max(og_model_acc))
print(best_og)
print(og_model_acc[best_og])

In [ ]:
worst_og = og_model_acc.index(min(og_model_acc))
print(worst_og)
print(og_model_acc[worst_og])

In [ ]:
average_og = np.average(og_model_acc)
print(average_og)

In [ ]:
# apply this section after specifying the chosen degree of pruning (threshold)
# saves the accuracy of the subsetted pruned Ensemble to ease comparison 
weights =[]
for model_weight in range(result.shape[0]):
        weights.append(result[model_weight][3]*100)
        
thresh = 0.131
weights = [1 if ele > thresh else 0 for ele in weights ]

larger_elements = [element for element in weights if element > 0]
number_of_elements = len(larger_elements)
print(number_of_elements)       
subset_model_acc = []
for model in range(segmented_last.shape[0]):
    if weights[model]:
        pred_temp = segmented_last[model]
        acc_temp = accuracy_score(y_train[40000:],pred_temp)
        subset_model_acc.append(acc_temp)

In [ ]:
print(subset_model_acc)
print(len(subset_model_acc))

In [ ]:
best_subset = subset_model_acc.index(max(subset_model_acc))
print(best_subset)
print(subset_model_acc[best_subset])

In [ ]:
worst_subset = subset_model_acc.index(min(subset_model_acc))
print(worst_subset)
print(subset_model_acc[worst_subset])

In [ ]:
average_subset = np.average(subset_model_acc)
print(average_subset)

In [ ]:
aggregate_og = np.average(segmented_last,axis=0)

aggregate_og = np.ceil(aggregate_og).astype(int)


acc_og = accuracy_score(y_train[40000:],aggregate_og)
print(acc_og) 

In [ ]:
# test data section read test data and apply ensemble voting with SOCP pruning as in the methods above
data = io.loadmat("RandModel_PRED_m_test.mat")
pred = data["PRED_m_test"]
print(pred.shape)
print(pred)

In [ ]:
# Ensemble run on test data (apply threshold parameter from validation part)
experiment_eval = []
for exp in range(result.shape[1]):
    print("hyper-parameter experiment{}".format(exp))
  
    weights =[]
    for model_weight in range(result.shape[0]):
        weights.append(result[model_weight][exp]*100)
    print(len(weights))
    
    thresh = 0.131
    weights = [1 if ele > thresh else 0 for ele in weights ]
    
    larger_elements = [element for element in weights if element > 0]
    number_of_elements = len(larger_elements)
    # voting
    voted_pred = [] 
    for model in range(pred.shape[0]):
        pred_temp = pred[model]
        if weights[model]:
            voted_pred.append(pred_temp)
    voted_pred = np.array(voted_pred)
    # aggregation
    aggregate = []
    for sample in range(voted_pred.shape[1]):
        sample_preds =[]
        for model in range(voted_pred.shape[0]):
            sample_preds.append(voted_pred[model][sample])
        
        aggregate.append(max(set(sample_preds), key = sample_preds.count))
       
    acc = accuracy_score(y_test,aggregate)
    experiment_eval.append(acc)

In [ ]:
# obtain test metrics 

best_experiment = experiment_eval.index(max(experiment_eval))
print(best_experiment)
print(experiment_eval[best_experiment])

In [ ]:
worst_experiment = experiment_eval.index(min(experiment_eval))
print(worst_experiment)
print(experiment_eval[worst_experiment])

In [ ]:
average_experiment = np.average(experiment_eval)
print(average_experiment)

In [ ]:
print(experiment_eval)

In [ ]:
# compare with full Ensemble on test data
og_model_acc = []
for model in range(pred.shape[0]):
    pred_temp = pred[model]
    acc_temp = accuracy_score(y_test,pred_temp)
    og_model_acc.append(acc_temp)

In [ ]:
best_og = og_model_acc.index(max(og_model_acc))
print(best_og)
print(og_model_acc[best_og])

In [ ]:
worst_og = og_model_acc.index(min(og_model_acc))
print(worst_og)
print(og_model_acc[worst_og])

In [ ]:
average_og = np.average(og_model_acc)
print(average_og)

In [ ]:
# subset on test for ease of comaprison
weights =[]
for model_weight in range(result.shape[0]):
        weights.append(result[model_weight][3]*100)
        
thresh = 0.131
weights = [1 if ele > thresh else 0 for ele in weights ]

larger_elements = [element for element in weights if element > 0]
number_of_elements = len(larger_elements)
print(number_of_elements)        
subset_model_acc = []
for model in range(pred.shape[0]):
    if weights[model]:
        pred_temp = pred[model]
        acc_temp = accuracy_score(y_test,pred_temp)
        subset_model_acc.append(acc_temp)

In [ ]:
best_subset = subset_model_acc.index(max(subset_model_acc))
print(best_subset)
print(subset_model_acc[best_subset])

In [ ]:
worst_subset = subset_model_acc.index(min(subset_model_acc))
print(worst_subset)
print(subset_model_acc[worst_subset])

In [ ]:
average_subset = np.average(subset_model_acc)
print(average_subset)